In [3]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *
from netgen.geom2d import SplineGeometry
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp

In [4]:
#4 by 4 square
# geo = SplineGeometry()
# geo.AddRectangle((-2,-2),(2,2))
# netgen_mesh = geo.GenerateMesh(maxh=4)
# mesh=Mesh(netgen_mesh)
meshwidth=1.0
netgen_mesh = unit_square.GenerateMesh(maxh=meshwidth)

mesh=Mesh(netgen_mesh) # 1.0
#help(mesh)
print(mesh.nface)
#help(mesh)

# for l in range(0):
#     mesh.Refine()
#     print(mesh.nface)
#     mesh.ngmesh.SplitPowellSabin()
#     print(mesh.nface)
#     meshwidth = meshwidth/2 

for f in mesh.faces:
    print (f, f.vertices)

for v in mesh.vertices:
    print(v, v.point)

#help( mesh.vertices)

#print(mesh.ne)

#help(netgen_mesh)
Draw(mesh)

#print(meshwidth)

elvol = Integrate(CoefficientFunction(1),mesh,element_wise=True)
h = [(2*vol)**(1/2) for vol in elvol]
print(min(h),max(h))



2
F0 (V0, V1, V2)
F1 (V0, V2, V3)
V0 (0.0, 0.0)
V1 (1.0, 0.0)
V2 (1.0, 1.0)
V3 (0.0, 1.0000000000000002)


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

0.999999999999998 0.9999999999999981


Set correct functionspaces in NGSolve, which will give the corresponding FE Elements, for l=0 its just H1 wihch will give the normal lagrangian elements, which for lowest order are the whitney 0 forms.

In [5]:
#set function space, for l=0 its normal lagrangian finite elements
H_h = Compress(H1(mesh, order = 1, complex=False)) # main function space
print(H_h.ndof)

#doestn like to draw gridfuntion on this
#mesh.ngmesh.SplitPowellSabin()

H_H = Compress(H1(mesh, order = 2, complex=False)) # high order Function space for Riesz representative
print(H_H.ndof)

help(H1)
#L_2 = L2(mesh, order=0)

# print("number of elements in mesh: ", mesh.ne)
# print("number of dofs per element: ", int(H_h.ndof/mesh.ne))

#gfu = GridFunction(H_h,multidim = H_h.ndof)
# for i in range(H_h.ndof):
#     gfu.vecs[i][i] = 0.2
# Draw(gfu, animate = True, min = 0, max = 0.2, order = 2, deformation = True, euler_angles =[-45,-6,25]);

#Draw(gfu)


#print(H_H.ndof)

4
9
Help on class H1 in module ngsolve.comp:

class H1(FESpace)
 |  An H1-conforming finite element space.
 |  
 |  The H1 finite element space consists of continuous and
 |  element-wise polynomial functions. It uses a hierarchical (=modal)
 |  basis built from integrated Legendre polynomials on tensor-product elements,
 |  and Jaboci polynomials on simplicial elements. 
 |  
 |  Boundary values are well defined. The function can be used directly on the
 |  boundary, using the trace operator is optional.
 |  
 |  The H1 space supports variable order, which can be set individually for edges, 
 |  faces and cells. 
 |  
 |  Internal degrees of freedom are declared as local dofs and are eliminated 
 |  if static condensation is on.
 |  
 |  The wirebasket consists of all vertex dofs. Optionally, one can include the 
 |  first (the quadratic bubble) edge basis function, or all edge basis functions
 |  into the wirebasket.
 |  
 |  Keyword arguments can be:
 |  
 |  order: int = 1
 |    or

In [6]:
#choose w_h and set its values on the nodes
w = sin(x**2) + cos(y**2)

# w_h = GridFunction(H_h) #set the nodes
# w_h.Set(w)

f_H = GridFunction(H_H, name="f_H") #high order 
#w_H.Set(w) #dont need to set the high order gridfunction

#Draw(w_H)

Create the galerkin matrices used to calculate the Riesz representative

In [7]:
#low order
u_h = H_h.TrialFunction() # define the trial function
v_h = H_h.TestFunction() # define the test function

#High order
u_H = H_H.TrialFunction() # define the trial function
v_H= H_H.TestFunction() # define the test function

# m = BilinearForm(H_H) #define m with the L2 inner product on the high order space
# m += u_H*v_H * dx 
# m.Assemble()

m = BilinearForm(H_h,symmetric_storage=True, symmetric=True) #define m with the L2 inner product on the low order space
m += u_h*v_h * dx 
m.Assemble()

a = BilinearForm(H_H, symmetric_storage=False, symmetric=True) # define the H1 inner product on the high order space
a += grad(u_H) * grad(v_H) * dx
a += u_H*v_H * dx 
a.Assemble()


#help(BilinearForm)

Approximate the dual norm, by the Riesz representive.
Get the Riesz Representive by solving a LGS, since we're approximating we set the space of the Riesz represntative in a higher order space (higher polynomial degree)

In [8]:
#create embedding matrix T

# a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)

# a_dest = BilinearForm(H_H)

# v_H_dual = v_H.Operator("dual")

# dS = dx(element_boundary=True)
# a_dest += u_H*v_H_dual*dx + u_H*v_H_dual*dS
# a_dest.Assemble()

# a_mixed += u_h*v_H_dual*dx + u_h*v_H_dual*dS
# a_mixed.Assemble()

# #build T
# t = a_dest.mat.Inverse() @ a_mixed.mat

############################# doing it directly as a L2 projection

a_mixed = BilinearForm(trialspace=H_h, testspace=H_H)
a_mixed += u_h*v_H*dx
a_mixed.Assemble()

print(a_mixed.mat.ToDense())

# a_dest = BilinearForm(H_H)
# a_dest += u_H*v_H*dx
# a_dest.Assemble()

#t = a_dest.mat.Inverse() @ a_mixed.mat

#w_H is the high order embedding of w_h
#w_H = GridFunction(H_H, name="w_H")
#w_H.vec.data = t * w_h.vec

#Draw(w_h)
#Draw(w_H)


######################## this works too

#this feels weird, find way to get the range from space H_h itself
#mixed = FESpace([H_h])

#emb = Embedding(H_H.ndof, mixed.Range(0))

#w_H.vec.data = emb * w_h.vec 

#Draw(w_H)

#results in same shape
# print(t.shape)
# print(emb.shape)



 0.166667 0.0416667 0.0833333 0.0416667
 0.0416667 0.0833333 0.0416667       0
 0.0833333 0.0416667 0.166667 0.0416667
 0.0416667       0 0.0416667 0.0833333
 -0.00833333 -0.00833333 -0.00416667       0
 -0.0166667 -0.00416667 -0.0166667 -0.00416667
 -0.00833333       0 -0.00416667 -0.00833333
 -0.00416667 -0.00833333 -0.00833333       0
 -0.00416667       0 -0.00833333 -0.00833333



In [9]:
#help(a.mat.Inverse(freedofs=H_H.FreeDofs()))
a_inv = a.mat.Inverse(freedofs=H_H.FreeDofs())
#print(H_h.FreeDofs())
m_inv = m.mat.Inverse(freedofs=H_h.FreeDofs())

#does the same as below
#f_H.vec.data = a_inv @ m.mat * w_H.vec
#Draw(f_H)

#solve for the riesz representative (f_H), wrong redo based on new notes
# temp = f_H.vec.CreateVector()
# temp.data = m.mat * w_H.vec
# f_H.vec.data = a_inv * temp
# Draw(f_H)



then plug the Riesz representive in the GEV problem and solve it, where the largest Eigenvalue will be the constant we're looking for

In [10]:
#create Matrices for GEV problem

#c is the low order galerkin matrix
c = BilinearForm(H_h, symmetric=True, symmetric_storage=True)
c += grad(u_h) * grad(v_h) * dx
c += u_h*v_h * dx 
c.Assemble()

#check norms
# temp3 = c.mat * w_h.vec
# w_h_norm = w_h.vec.InnerProduct(temp3)
# print("H(div) norm of w_h:", sqrt(w_h_norm)) #norm of w_h in H(div,

# temp4 = a.mat * f_H.vec
# f_H_norm = f_H.vec.InnerProduct(temp4)
# print("H(div) norm of f_H, the riesz representative of w_h:", sqrt(f_H_norm)) #norm of the riesz representative

#old b
#b = t.T @ m.mat.T @ a_inv @ m.mat @ t
#new b, uses amixed instead of t
b = c.mat @m_inv @ a_mixed.mat.T @ a_inv @ a_mixed.mat @ m_inv @ c.mat

#evem wiith complex space no complex values
#print(a_mixed.mat)

# C = c.mat.ToDense().NumPy()
# M = m.mat.ToDense().NumPy()
# M_inv = m_inv.ToDense().NumPy()
# A_inv = a_inv.ToDense().NumPy()
# T = a_mixed.mat.ToDense().NumPy()

# #M == M.T
# # old B 
# #B = T.T @ M.T @ A_inv @ M @ T
# #new B
# B = b.ToDense().NumPy() #C.T @ M_inv.T @ T.T @ A_inv @ T @ M_inv @ C
# B_d = C.T @ M_inv.T @ T.T @ A_inv @ T @ M_inv @ C

print(b.ToDense())

#practically 0 everywhere
#print(B - B_d)

 1.36196 -0.614264  0.1999 -0.614264
 -0.614264  1.2491 -0.614264 0.146093
  0.1999 -0.614264 1.36196 -0.614264
 -0.614264 0.146093 -0.614264  1.2491



In [11]:
#solve GEV using the scipy solver s

#currently still not correct

#y = sp.linalg.eigvals(C,B)
x = sp.sparse.linalg.eigs(C, k=1, M=B, which='SM', return_eigenvectors=False)
#print("maximal Eigenvalue:",max(y))
#print("minimal Eigenvalue:",min(y))
print("minimal Eigenvalue (sparse solver):",x)


NameError: name 'C' is not defined

In [ ]:
w_h = GridFunction(H_h)

tmp1 = w_h.vec.CreateVector()
tmp2 = w_h.vec.CreateVector()
def matvec_b(v):
    tmp1.data = v
    tmp2.data = b * tmp1
    return tmp2.FV().NumPy()

tmp3 = w_h.vec.CreateVector()
tmp4 = w_h.vec.CreateVector()
def matvec_c(v):
    tmp3.FV().NumPy()[:] = v
    tmp4.data = c.mat * tmp3
    return tmp4.FV().NumPy()

B_ = sp.sparse.linalg.LinearOperator((b.height,b.width), matvec_b)
C_ = sp.sparse.linalg.LinearOperator((c.mat.height,c.mat.width), matvec_c)
x = sp.sparse.linalg.eigs(C_, k=1, M=B_, which='SM', return_eigenvectors=False)

print(x)

[0.76720499+0.j]


h = 1, minEV = 0.76720499
h = 0.5, minEV = 0.76004237
h = 0.25, minEV = 0.59529111
h = 0.125, minEV = 0.52461719 
h = 0.0625, minEV = 0.50758144
h = 0.03125, minEv = 0.50339601
h = 0.015625 , minEV = 0.50235468